<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

# Set up a flowline model run with known boundary conditions

This notebook builds upon the ``flowlime_model.ipynb`` example notebook, but this time uses known input data for the model. More importantly (and digging a bit deeper in model structure), it illustrates how to write a custom mass-balance model.

In [ ]:
# The commands below are just importing the necessary modules and functions
# Plot defaults
%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9, 6)  # Default plot size
# Scientific packages
import pandas as pd
import numpy as np
# Constants
from oggm.cfg import SEC_IN_YEAR, A, RHO
# OGGM models
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline
from oggm.core.massbalance import MassBalanceModel

## Read the bed topography data

In [ ]:
df = pd.read_csv('/home/mowglie/bedrock.csv', index_col=0)
df.plot();

This is an ideal bedrock taken from the OGGM [numerics test suite](https://github.com/OGGM/oggm/blob/master/oggm/tests/test_numerics.py). Now we can create a flowline from these data. Let's give our glacier a rectangular bed shape for this example (not very realistic but easier to interpret): 

In [ ]:
map_dx = 100.  # units: meter, obtained from the dataframe's index above
# The units of widths is in "grid points", i.e. 3 grid points = 300 m in our case
widths = np.zeros_like(df['z_bedrock']) + 3.
# Define our bed
init_flowline = RectangularBedFlowline(surface_h=df['z_ice'].values, 
                                       bed_h=df['z_bedrock'].values, 
                                       widths=widths, 
                                       map_dx=map_dx)

Again, this ``Flowline`` object has some nice attributes for you to check out:

In [ ]:
print('Glacier length:', init_flowline.length_m)
print('Glacier area:', init_flowline.area_km2)
print('Glacier volume:', init_flowline.volume_km3)

## Make a custom mass-balance model

To make a custom mass-balance model to OGGM, you'll need to comply to a relatively simple interface: you should create a new class inheriting from the ``oggm.core.massbalance.MassBalanceModel`` class. Your task will be to implement one single method: ``get_annual_mb`` (and ``get_monthly_mb`` if you'd like to use a monthly resolution for the dynamical model, which is not necessarily a good idea).

In [ ]:
class SimpleRandomMassBalance(MassBalanceModel):
    """Custom mass-balance model for OGGM.
    
    It is a normally distributed random mass-balance with a linear gradient.
    
    You'll need to add your own code here for more elaborated models.
    """

    def __init__(self, avg_ela_h=2500, sigma_ela_h=300, grad=3, seed=None):
        """ Initialize.

        Parameters5
        ----------
        avg_ela_h: float
            Average equilibrium line altitude (units: [m])
        sigma_ela_h : float
            Standard deviation ([m]) of the ela annual variability
        grad: float
            Mass-balance gradient (unit: [mm w.e. yr-1 m-1])
        """
        super(SimpleRandomMassBalance, self).__init__()
        self.avg_ela_h = avg_ela_h
        self.sigma_ela_h = sigma_ela_h
        self.grad = grad
        self.rng = np.random.RandomState(seed)

    def get_annual_mb(self, heights, year=None):
        """Annual mass-balance at given altitude(s) for a moment in time.

        Units: [m s-1], or meters of ice per second

        Note: `year` is optional because some simpler models have no time
        component.
        
        Important Note: the unit is meter of ICE per second. If your mass
        balance is in mm we (or kg m-2), a good way to convert it would be:
        ``mb_ice = mb_mm_we / cfg.RHO`` where cfg.RHO is the density of 
        ice (900 kg m-3 in OGGM).

        Parameters
        ----------
        heights: ndarray
            the atitudes at which the mass-balance will be computed
        year: float, optional
            the time

        Returns
        -------
        the mass-balance (same dimension as `heights`) (units: [m s-1])
        """
        ela = self.rng.normal(self.avg_ela_h, self.sigma_ela_h)
        mb = (np.asarray(heights) - ela) * self.grad
        return mb / SEC_IN_YEAR / RHO

In [ ]:
# Initialise it
mb_model = SimpleRandomMassBalance()

The mass-balance model gives you the mass-balance for any altitude you want, in units [m s$^{-1}$]. Let us compute the *annual* mass-balance along the glacier profile:

In [ ]:
annual_mb = mb_model.get_annual_mb(init_flowline.surface_h) * SEC_IN_YEAR

In [ ]:
# Plot it
plt.figure();
plt.plot(annual_mb, init_flowline.surface_h, color='C2', label='Mass-balance')
plt.xlabel('Annual mass-balance (m yr-1)')
plt.ylabel('Altitude (m)')
plt.legend(loc='best');

Let's compute the point mass-balance at the average ELA altitude:

In [ ]:
time = np.arange(500)
annual_mb = [mb_model.get_annual_mb(2500, year=t)*SEC_IN_YEAR for t in time]
plt.figure()
plt.plot(time, annual_mb)
plt.xlabel('Year')
plt.ylabel('Annual mass-balance at ELA (m yr-1)');

### Model run

Now that we have all the ingredients to run the model, we just have to initialize it:

In [ ]:
# The model requires the initial glacier bed, a mass-balance model, and an initial time (the year y0)
model = FluxBasedModel(init_flowline, mb_model=mb_model, y0=0., inplace=False, time_stepping='conservative')

In [ ]:
# Year 0 to 800 in 5 years step
yrs = np.arange(0, 801, 5)
# Array to fill with data
nsteps = len(yrs)
length = np.zeros(nsteps)
vol = np.zeros(nsteps)
surface_h_ts = []
# Loop
for i, yr in enumerate(yrs):
    model.run_until(yr)
    length[i] = model.length_m
    vol[i] = model.volume_km3
    surface_h_ts.append(model.fls[-1].surface_h.copy())
# I store the final results for later use
simple_glacier_h = model.fls[-1].surface_h

We can now plot the evolution of the glacier length and volume with time:

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
ax1.plot(yrs, length);
ax1.set_xlabel('Years')
ax1.set_ylabel('Length (m)');
ax2.plot(yrs, vol);
ax2.set_xlabel('Years')
ax2.set_ylabel('Volume (km3)');
plt.tight_layout();

Let's make a widget just for fun:

In [ ]:
import ipywidgets
f, ax = plt.subplots(figsize=(9, 6))
ax.plot(model.fls[-1].bed_h, color='k', label='Bedrock')
gl = ax.plot(surface_h_ts[0], label='Glacier')[0]
plt.tight_layout()
plt.legend()
def plot_func(freq):
    gl.set_ydata(surface_h_ts[freq//5])
    f.canvas.draw()
freq = ipywidgets.IntSlider(value=0, min=0, max=800, step=10, description='Year:')
ipywidgets.interact(plot_func, freq=freq);